### Twitter Archive to CSV Converter

Old archive format

In [1]:
import os                           # For listing directories
import json                         # For JSON-structured file reader
from operator import itemgetter     # For sorting
import csv                          # For loading to target CSV

# Path to the tweet archives folder (data)
path = '..//..//Data//tweets_old_format'

# Container for the end result
tweets = []

# Walk through the archived files
for(root, dirs, files) in os.walk(path):
    # Iterate the files
    for file in files:
        # Open each file
        with open (f'{path}//{file}') as f:
            # Skip the first line (variable assignment)
            next(f)
            for tweet in json.load(f):
                # Create row variable
                temp_tweet = []

                # Tweet ID
                temp_tweet.append(tweet["id_str"])
                # Tweet Link
                temp_tweet.append(f'https://twitter.com/{tweet["user"]["screen_name"]}/status/{tweet["id_str"]}')
                # Tweet ID
                temp_tweet.append(tweet["created_at"])
                # Tweet Content
                temp_tweet.append(tweet["text"])

                # Completed, add to master list
                tweets.append(temp_tweet)

# Redo the order since content of data files are unordered
tweets = sorted(tweets, key=itemgetter(0))

# Write into the target CSV
with open('..//..//Data//Private Data//Tweet Collection.csv', 'w', newline='',encoding='utf-8') as file:
    # Add proper header
    csv_header = csv.DictWriter(file, delimiter=',', fieldnames=['Tweet ID', 'Tweet Link', 'Date','Tweet Content'])
    csv_header.writeheader()

    # Write file
    csv_writer = csv.writer(file)
    for i in tweets:
        csv_writer.writerow(i)

print('Done!')

Done!


New archive format

In [2]:
import json                         # For JSON-structured file reader
from operator import itemgetter     # For sorting
import csv                          # For loading to target CSV
from datetime import datetime
from sensitivevariable import twitter_username

# Path to the tweet archives folder (data)
path = '..//..//Data//Private Data//tweets_new_format.js'

# Twitter/X Username (manual input)
# username = 'elonmusk
username = twitter_username

# Container for the end result
tweets = []

# Load file
with open (path,encoding='utf-8') as f:
    loaded_file = f.read()

    # Skip the variable assignment at the beginning of the file
    file_in_string = loaded_file[loaded_file.index('['):]

    # Load the string into JSON format
    data = json.loads(file_in_string)

    # Loop through the entries
    for tweet in data:
        # Create row variable
        tweet_row = []

        # Tweet ID
        tweet_row.append(tweet['tweet']['id'])

        # Tweet Date - Retrieve the date, but convert it from '%a %b %d %H:%M:%S %z %Y' format to '%Y-%m-%d %H:%M:%S %z %Z' format.
        tweet_row.append(datetime.strptime(tweet['tweet']['created_at'], '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d %H:%M:%S %z %Z'))

        # Tweet Link
        tweet_row.append(f'https://twitter.com/{username}/status/{tweet["tweet"]["id"]}')

        # Tweet Content
        tweet_row.append(tweet['tweet']['full_text'])

        # Mentioned Handles
        mentioned_handles = []
        for h in tweet['tweet']['entities']['user_mentions']:
            mentioned_handles.append('@'+h['screen_name'])
        tweet_row.append(', '.join(mentioned_handles))

        # Via/Platform - Manually capture the content between first '>' and second '<'
        tweet_row.append(tweet['tweet']['source'][tweet['tweet']['source'].index('>')+1:tweet['tweet']['source'][1:].index('<')+1])

        # Favorite Count
        tweet_row.append(tweet['tweet']['favorite_count'])

        # Retweet Count
        tweet_row.append(tweet['tweet']['retweet_count'])

        # Completed, add to master list
        tweets.append(tweet_row)


# Redo the order in case tweets are unordered
tweets = sorted(tweets, key=itemgetter(1))
print(f'Generating CSV for {len(tweets):,} tweets...')
# Write into the target CSV
with open('..//..//Data//Private Data//Tweet Collection v2.csv', 'w', newline='',encoding='utf-8') as file:
    # Add proper header
    csv_header = csv.DictWriter(file, delimiter=',', fieldnames=['Tweet ID','Tweet Date','Tweet Link','Tweet Content','Mentioned Handle(s)/Username(s)','Via','Favorite Count','Retweet Count'])
    csv_header.writeheader()

    # Write file
    csv_writer = csv.writer(file)
    for i in tweets:
        csv_writer.writerow(i)

print('Done!')

Generating CSV for 15,252 tweets...
Done!
